# Building the Transcript Database

We want to build a database to categorize all the transcripts available. We want to capture the campaign, arc, and episode information as much as possible so that we have that metadata in the future.

In [51]:
from bs4 import BeautifulSoup
import pandas as pd

import urllib
import urllib.request

In [2]:
transcripts_url = 'https://criticalrole.fandom.com/wiki/Transcripts'

with urllib.request.urlopen(transcripts_url) as response:
  html = response.read()

transcripts = BeautifulSoup(html, 'html.parser')

We are going to use the structure of the transcripts page to get all the data we need. We discovered this structure by manual inspection. The transcripts are organized by campaign (or miscellaneous section for non-campaign episodes), arc within a campaign (or subsection for miscellaneous), and then multiple episodes. We can read through the html and keep track of which campaign/section and arc/subsection we are in to number the episodes.

In [47]:
main_div = transcripts.find_all('div', {'class': 'mw-parser-output'})[0]

In [63]:
current_section       = None
current_section_no    = 0
current_subsection    = None
current_subsection_no = 0
current_episode_no    = 0
transcript_data       = []
for child in main_div.find_all():
    match child.name:
        case 'h2':
            child_span = child.find('span', {'class': 'mw-headline'})
            if child_span is not None:
                current_section       = child_span.text
                current_section_no   += 1
                current_subsection_no = 0
                current_episode_no    = 0
        case 'h3':
            child_span = child.find('span', {'class': 'mw-headline'})
            if child_span is not None:
                current_subsection     = child_span.text
                current_subsection_no += 1
                current_episode_no     = 0
        case 'a':
            if child.text == 'Transcript':
                current_episode_no += 1
                transcript_data.append([
                    current_section_no, current_subsection_no, current_episode_no,
                    current_section, current_subsection, child['href'][6:-11],
                    'https://criticalrole.fandom.com' + child['href']
                ])

transcript_df = pd.DataFrame(
    transcript_data,
    columns = ['section_no', 'subsection_no', 'episode_no', 'section', 'subsection', 'episode', 'link']
)
transcript_df

,section_no,subsection_no,episode_no,section,subsection,episode,link
0,1,1,1,Campaign 1: Vox Machina,Arc 1: Kraghammer and Vasselheim,Arrival_at_Kraghammer,https://criticalrole.fandom.com/wiki/Arrival_a...
1,1,1,2,Campaign 1: Vox Machina,Arc 1: Kraghammer and Vasselheim,Into_the_Greyspine_Mines,https://criticalrole.fandom.com/wiki/Into_the_...
2,1,1,3,Campaign 1: Vox Machina,Arc 1: Kraghammer and Vasselheim,Strange_Bedfellows,https://criticalrole.fandom.com/wiki/Strange_B...
3,1,1,4,Campaign 1: Vox Machina,Arc 1: Kraghammer and Vasselheim,Attack_on_the_Duergar_Warcamp,https://criticalrole.fandom.com/wiki/Attack_on...
4,1,1,5,Campaign 1: Vox Machina,Arc 1: Kraghammer and Vasselheim,The_Trick_about_Falling,https://criticalrole.fandom.com/wiki/The_Trick...
...,...,...,...,...,...,...,...
464,6,1,13,Miscellaneous,Candela Obscura,Candela_Obscura_Live_-_The_Circle_of_the_Silve...,https://criticalrole.fandom.com/wiki/Candela_O...
465,6,3,1,Miscellaneous,Undeadwood,"UnDeadwood_Part_I:_Stay_Close,_Reverend",https://criticalrole.fandom.com/wiki/UnDeadwoo...
466,6,3,2,Miscellaneous,Undeadwood,UnDeadwood_Part_II:_God_Don%27t_Play_Cards,https://criticalrole.fandom.com/wiki/UnDeadwoo...
467,6,3,3,Miscellaneous,Undeadwood,UnDeadwood_Part_III:_I_Got_My_Wish,https://criticalrole.fandom.com/wiki/UnDeadwoo...


In [64]:
pd.crosstab(transcript_df['section'], transcript_df['subsection'])

subsection,Arc 1: Come Together,Arc 1: Jrusar,Arc 1: Kraghammer and Vasselheim,Arc 2: Ruidus Rising,Arc 2: The Bad Guys,Arc 2: The Briarwoods,Arc 3: Separations and Explorations,Arc 3: The Bright Queen's Favor,Arc 3: The Chroma Conclave,Arc 4: Swords and Angels,Arc 4: Taryon Darrington,Arc 5: Family Ties,Arc 5: Vecna,Arc 6: Weird Magic,Campaign Three Arc 4,Candela Obscura,Undeadwood
section,,,,,,,,,,,,,,,,,
Campaign 1: Vox Machina,0,0,23,0,0,15,0,0,46,0,15,0,16,0,0,0,0
Campaign 2: The Mighty Nein,35,0,0,0,12,0,0,22,0,22,0,21,0,29,0,0,0
Campaign Three: Bells Hells,0,23,0,28,0,0,24,0,0,0,0,0,0,0,23,0,0
Exandria Unlimited,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0
Miscellaneous,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,4
Specials,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,0,0


We can see that Exandria Unlimited and Specials have the subsection of 'Campaign Three Arc 4' because they don't have subsections. We can update those to be the same as the section.

In [65]:
transcript_df['subsection'] = transcript_df.apply(
    lambda x: x['section'] if ['Exandria Unlimited', 'Specials'].count(x['section']) > 0 else x['subsection'],
    axis=1)
pd.crosstab(transcript_df['section'], transcript_df['subsection'])

subsection,Arc 1: Come Together,Arc 1: Jrusar,Arc 1: Kraghammer and Vasselheim,Arc 2: Ruidus Rising,Arc 2: The Bad Guys,Arc 2: The Briarwoods,Arc 3: Separations and Explorations,Arc 3: The Bright Queen's Favor,Arc 3: The Chroma Conclave,Arc 4: Swords and Angels,Arc 4: Taryon Darrington,Arc 5: Family Ties,Arc 5: Vecna,Arc 6: Weird Magic,Campaign Three Arc 4,Candela Obscura,Exandria Unlimited,Specials,Undeadwood
section,,,,,,,,,,,,,,,,,,,
Campaign 1: Vox Machina,0,0,23,0,0,15,0,0,46,0,15,0,16,0,0,0,0,0,0
Campaign 2: The Mighty Nein,35,0,0,0,12,0,0,22,0,22,0,21,0,29,0,0,0,0,0
Campaign Three: Bells Hells,0,23,0,28,0,0,24,0,0,0,0,0,0,0,23,0,0,0,0
Exandria Unlimited,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,0,0
Miscellaneous,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,4
Specials,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,0


Check to make sure that the episode numer is resetting correctly.

In [68]:
transcript_df.groupby(['section', 'subsection']).agg({'episode_no': ['min', 'max']})

episode_no    
                                                                       min max
section                     subsection                                        
Campaign 1: Vox Machina     Arc 1: Kraghammer and Vasselheim             1  23
                            Arc 2: The Briarwoods                        1  15
                            Arc 3: The Chroma Conclave                   1  46
                            Arc 4: Taryon Darrington                     1  15
                            Arc 5: Vecna                                 1  16
Campaign 2: The Mighty Nein Arc 1: Come Together                         1  35
                            Arc 2: The Bad Guys                          1  12
                            Arc 3: The Bright Queen's Favor              1  22
                            Arc 4: Swords and Angels                     1  22
                            Arc 5: Family Ties                           1  21
                            Arc 6: Weird Magic                           1  29
Campaign Three: Bells Hells Arc 1: Jrusar                                1  23
                            Arc 2: Ruidus Rising                         1  28
                            Arc 3: Separations and Explorations          1  24
                            Campaign Three Arc 4                         1  23
Exandria Unlimited          Exandria Unlimited                           1  14
Miscellaneous               Candela Obscura                              1  13
                            Undeadwood                                   1   4
Specials                    Specials                                     1  84

In [69]:
transcript_df.to_csv('../../data/transcript_database.csv', index = False)